在LangGraph中，State是一个共享的数据结构，用于在整个工作流（Workflow）或智能体（Agent）执行过程中存储和传递信息。
- State是一个全局变量容器，是“记忆中枢”，所有节点（Node）都可以读取和更新它。
- State 是有类型的（通常用 Pydantic 模型或 TypedDict 定义），确保数据结构清晰、可维护。
- 整个工作流的演进，本质上就是 State 不断被节点读取和修改的过程。

# State组成

状态由图的模式（Schema）以及归约器函数（reducer functions）组成：
- 图的模式（Schema）定义了状态的结构和类型
- 归约器函数（reducer functions）用于指定如何将更新应用到状态上

## State Schema

根据LangChain官方的推荐，推荐Schema定义的方式是TypedDict，跟Pydantic和dataclass的区别前面的可能我们已经介绍过，这里不再赘述。

In [1]:
from typing_extensions import TypedDict

class State(TypedDict):
    foo: int
    bar: list[str]

这里为了演示，先引入LangGraph的使用，后面会详细介绍。

In [2]:
# 定义langgraph的节点和图
from langgraph.graph import StateGraph, START, END

def node_a(state: State) -> dict:
    print(f"[node_a] 收到 state: {state}")
    # 将 foo 加 10，并向 bar 添加一个字符串
    return {
        "foo": state["foo"] + 10,
        "bar": ["来自 node_a 的消息"]
    }

def node_b(state: State) -> dict:
    print(f"[node_b] 收到 state: {state}")
    # 再次修改 foo，并扩展 bar 列表
    return {
        "foo": state["foo"] * 2,
        "bar": state["bar"] + ["来自 node_b 的消息"]
    }

# 构建图，传入预先定义好的State
builder = StateGraph(State)

# 添加节点
builder.add_node("a", node_a)
builder.add_node("b", node_b)

# 设置入口和流程
builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("b", END)

# 编译图
graph = builder.compile()


In [ ]:
# 运行图，观察state变化
initial_state = {"foo": 5, "bar": []}

# invoke、stream等方法第一个参数为State的实例
final_state = graph.invoke(initial_state)

print("\n最终 State:")
print(final_state)

[node_a] 收到 state: {'foo': 5, 'bar': []}
[node_b] 收到 state: {'foo': 15, 'bar': ['来自 node_a 的消息']}

最终 State:
{'foo': 30, 'bar': ['来自 node_a 的消息', '来自 node_b 的消息']}


## 归约器函数

归约器函数（Reducer Function）是理解节点更新如何应用到状态（State）中的关键。状态中的每个键（key）都拥有自己独立的归约函数。如果没有显式指定归约函数，则默认认为对该键的所有更新都应直接覆盖其当前值。

使用了Annotated类型为第二个键（bar）指定一个归约函数（operator.add）:

In [7]:
## 归约函数
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

此时如果节点再去修改bar值，默认会追加到列表之中，这就是归约器起了作用。

# 内置的归约器（Reducer）

## Overwrite

在某些情况下，你可能希望绕过归约函数（reducer），直接覆盖状态值。LangGraph 为此提供了 Overwrite 类型，它的作用非常明确：无论旧值是什么，一律用新值完全替换（覆盖），不进行任何合并或累加，适合在某些节点中直接修改状态值。

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from operator import add
from langgraph.types import Overwrite  # ← 关键导入

# 定义 State
class State(TypedDict):
    # 默认行为：列表会追加
    latest_status: Annotated[list[str], add]
    log_messages: Annotated[list[str], add] 

def node_a(state: State) -> dict:
    print(f"[node_a] 输入 state: {state}")
    return {
        "latest_status": ["处理中..."],      # 正常追加
        "log_messages": ["步骤A完成"]
    }

def node_b(state: State) -> dict:
    print(f"[node_b] 输入 state: {state}")
    return {
        "latest_status": Overwrite(["已完成"]),   # 覆盖 node_a 中的值
        "log_messages": ["步骤B完成"]
    }

# 构建图
builder = StateGraph(State)
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("b", END)

graph = builder.compile()

# 初始状态
initial = {"latest_status": ["初始状态"], "log_messages": []}
result = graph.invoke(initial)

print("\n最终 State:")
print(result)

[node_a] 输入 state: {'latest_status': ['初始状态'], 'log_messages': []}
[node_b] 输入 state: {'latest_status': ['初始状态', '处理中...'], 'log_messages': ['步骤A完成']}

最终 State:
{'latest_status': ['已完成'], 'log_messages': ['步骤A完成', '步骤B完成']}


## add_messages

在许多情况下，将先前的对话历史以消息列表的形式存储在图的状态中非常有用。为此，我们可以在图状态中添加一个键（即通道），用于存储消息对象（Message objects）的列表，并为其标注一个归约函数（reducer function）。

langgraph提供了一个名为add_messages的归约器，会在每次接收到 messages 通道上的状态更新时，尝试将消息反序列化为 LangChain 的 Message 对象并进行去重（通过唯一的消息ID实现，如果不手动指定，add_messages会自动创建ID），最终追加到messages上，相比使用list和add的组合更加智能。

> 注意：只有在消息经过 add_messages reducer 处理后，才会被赋予 ID。如果你在图外部手动创建消息且未设 id，此时 msg.id 为 None。

In [14]:
from langchain.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [17]:
from langchain.messages import HumanMessage, AIMessage, ToolMessage

def user_ask(state: GraphState) -> dict:
    """模拟用户提问"""
    print(f"[user_ask] 当前消息数: {len(state['messages'])}")
    # 返回一条用户消息
    return {"messages": [HumanMessage(content="今天天气怎么样？")]}

def ai_respond(state: GraphState) -> dict:
    """模拟 AI 回复（含工具调用）"""
    print(f"[ai_respond] 收到消息: {state['messages'][-1].content}")
    # 返回一条 AI 消息（模拟需要调用工具）
    return {
        "messages": [
            AIMessage(
                content="",
                tool_calls=[{"name": "get_weather", "args": {"loc": "北京"}, "id": "t1"}]
            )
        ]
    }

def tool_execute(state: GraphState) -> dict:
    """模拟工具执行结果"""
    last_ai_msg = state["messages"][-1]
    if hasattr(last_ai_msg, 'tool_calls') and last_ai_msg.tool_calls:
        tool_call = last_ai_msg.tool_calls[0]
        result = f"宁波天气晴朗，25°C"
        return {
            "messages": [
                ToolMessage(content=result, tool_call_id=tool_call["id"])
            ]
        }
    return {"messages": []}

def final_answer(state: GraphState) -> dict:
    """最终回答"""
    print(f"[final_answer] 工具结果: {state['messages'][-1].content}")
    return {
        "messages": [AIMessage(content="今天宁波天气晴朗，气温25摄氏度。")]
    }

# 3. 构建图
builder = StateGraph(GraphState)

builder.add_node("ask", user_ask)
builder.add_node("think", ai_respond)
builder.add_node("tool", tool_execute)
builder.add_node("answer", final_answer)

builder.add_edge(START, "ask")
builder.add_edge("ask", "think")
builder.add_edge("think", "tool")
builder.add_edge("tool", "answer")
builder.add_edge("answer", END)

graph = builder.compile()

# 4. 运行（初始状态为空消息列表）
result = graph.invoke({"messages": []})

print("\n=== 最终所有消息 ===")
for msg in result["messages"]:
    msg.pretty_print()

[user_ask] 当前消息数: 0
[ai_respond] 收到消息: 今天天气怎么样？
[final_answer] 工具结果: 宁波天气晴朗，25°C

=== 最终所有消息 ===
================================ Human Message =================================

今天天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  get_weather (t1)
 Call ID: t1
  Args:
    loc: 北京
================================= Tool Message =================================

宁波天气晴朗，25°C
================================== Ai Message ==================================

今天宁波天气晴朗，气温25摄氏度。


# 内置的State

由于在状态中保存消息列表是非常常见的做法，因此 LangGraph 提供了一个预构建的状态类 MessagesState，可以方便地使用消息。MessagesState 仅定义了一个名为 messages 的键，其值是一个 AnyMessage 对象的列表，并默认使用 add_messages 作为归约函数（reducer）。

In [ ]:
from langgraph.graph import MessagesState

class State(MessagesState):
    """
    状态类，继承自MessagesState，默认已有messages键，额外添加了自定义的documents键
    """
    documents: list[str]

在LangChain中，则是预定义了AgentState类，作用与MessagesState一样，默认已有messages键。

In [19]:
from langchain.agents import AgentState

class CustomAgentState(AgentState):  
    user_id: str
    preferences: dict

在后续的课程中，我们的自定义state都会继承这两个类，以获得messages键的功能。

# 总结

无论是LangGraph的图还是LangChain的Agent，都是使用State来管理短期记忆，具体是通过messages键来维护对话历史，通过扩展额外的键来满足运行的需求。同时，State还是checkpoint的核心部分。
